In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
def identity_block(x, filter):

    x_skip = x

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)
    x =  keras.layers.Activation('relu')(x)

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)

    x =  keras.layers.Add()([x, x_skip])     
    x =  keras.layers.Activation('relu')(x)
    return x

In [ ]:
def convolutional_block(x, filter):

    x_skip = x

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)
    x =  keras.layers.Activation('relu')(x)

    x =  keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x =  keras.layers.BatchNormalization(axis=3)(x)

    x_skip =  keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)

    x =  keras.layers.Add()([x, x_skip])     
    x =  keras.layers.Activation('relu')(x)
    return x

In [ ]:
def ResNet18(shape = (96, 96, 3), classes = 1):

    x_input = keras.layers.Input(shape)
    x = keras.layers.ZeroPadding2D((3, 3))(x_input)

    x = keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    block_layers = [2,2,2,2]
    filter_size = 64

    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)

    x = keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(512, activation = 'relu')(x)
    x = keras.layers.Dense(classes, activation = 'sigmoid')(x)
    model = keras.models.Model(inputs = x_input, outputs = x, name = "ResNet18")
    model.compile(loss='binary_crossentropy', optimizer=lr, metrics=['accuracy'])
    return model

In [ ]:
data_dir_train = 'Dataset/train/'
data_dir_test = 'Dataset/test/'

img_size = 256
input_shape = (img_size, img_size, 3)
target_size =(img_size,img_size)
batch_size = 32
num_classes = 1

test_num = "100"

compile_optimizer = "adam"
compile_loss = "binary_crossentropy"
lr = keras.optimizers.Adam(learning_rate=0.0005)

csv_path = f'Results/Dataset_ResNet18_{batch_size}_{img_size}_Test_{test_num}.csv'
save_model_path = f"Models/Dataset_ResNet18_{batch_size}_{img_size}_Test_{test_num}.h5"

In [ ]:
training_data = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 40,
        width_shift_range = 0.25,
        height_shift_range = 0.25,
        shear_range = 0.25,
        zoom_range = 0.3,
        brightness_range= [0.6,0.9],
        horizontal_flip = True,
        fill_mode = 'nearest',
        validation_split = 0.2,
        )

test_data = ImageDataGenerator(
        rescale = 1./255,
        )

In [ ]:
train_ds = training_data.flow_from_directory(
    directory=data_dir_train,
    shuffle = True,
    seed = 2,
    class_mode = 'binary',
    target_size=target_size,
    batch_size=batch_size,
    subset="training",
   )

validation_ds = training_data.flow_from_directory(
    directory=data_dir_train,
    shuffle = True,
    seed = 2,
    class_mode = 'binary',
    target_size=target_size,
    batch_size=batch_size,
    subset="validation",
    )

test_ds = test_data.flow_from_directory(
    directory=data_dir_test,
    target_size=target_size,
    shuffle = False,
    )

In [ ]:
model = ResNet18(input_shape)
model.summary()

In [ ]:
#checkpoint = ModelCheckpoint(save_model_path, monitor='val_accuracy', save_best_only=True, save_weights_only=True, mode='auto')

#early = EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

#history = model.fit(train_ds, epochs=300, callbacks=[checkpoint,early], validation_data = validation_ds)

In [ ]:
model.load_weights(save_model_path)
model.evaluate(validation_ds)
#df = pd.DataFrame(history.history)
#df.plot(figsize=(10,8))

In [ ]:
pred = model.predict(test_ds, steps = len(test_ds), verbose=1)
cl = np.round(pred)
classes_prediction = []
for prediction in cl[:,0]:
    if (prediction == 0.0):
        classes_prediction.append('Cracked')
    elif(prediction == 1.0):
        classes_prediction.append('Uncracked')

filenames=test_ds.filenames
results=pd.DataFrame({"file":filenames,"prediction":pred[:,0], "class":classes_prediction})

In [ ]:
results.to_csv(csv_path)
#import jinja2
#results.style

In [ ]:
import cv2 
rez = pd.read_csv(csv_path)
plt.figure(figsize=(40,28))
for i in range(56):
    img_path = str(rez['file'][i])
    image_path = data_dir_test + 'test/' + img_path[5:]
    #print(image_path)
    class_pred = str(rez['class'][i])
    img = cv2.imread(image_path)
    plt.subplot(7,8, i+1)
    plt.imshow(img)
    plt.title(class_pred)

In [ ]:
print(classes_prediction.count("Cracked"))